In [12]:
import oandapyV20
from oandapyV20 import API
import oandapyV20.endpoints.pricing as pricing
import oandapyV20.endpoints.accounts as accounts
import oandapyV20.endpoints.orders as orders
import oandapyV20.endpoints.trades as trades 
import oandapyV20.endpoints.positions as positions
import oandapyV20.endpoints.transactions as trans
import oandapyV20.endpoints.instruments as instruments
import pandas as pd

import json

from oandapyV20.exceptions import V20Error
from oandapyV20.endpoints.pricing import PricingStream
import numpy as np
import datetime
def convert_to_df():
    print()
    print()
    print()
    print(instrument,granularity,count,percentile_test)
    print('START:',start,' END:',end)
    if positive:
        negative = False
    else:
        negative = True
    params = {#"count":count,
              "granularity": granularity,
              "from": str(start.isoformat('T')),
              "to": str(end.isoformat('T'))
                 }
    print('PARAMS',params)
    print(instrument,type(instrument))
    r = instruments.InstrumentsCandles(instrument=instrument,
                                      params=params)
    client.request(r)
    dct = r.response
    lst = dct['candles']
    if len(lst) == 0:
        print('NO RESULT RETRUNING EMPTY DF...')
        return pd.DataFrame([0])
    lst = []
    i = 0
    
    for candle in dct['candles']:
        #IF CANDLE IS BULLISH
        if (float(candle['mid']['c']) - float(candle['mid']['o'])) > 0:
            max_vector = (float(candle['mid']['h']) - float(candle['mid']['l']))
            
        #IF CANDLE IS BEARISH
        else:
            max_vector = -1 * (float(candle['mid']['h']) - float(candle['mid']['l']))
            
        if i == 0:
            prev_delta = 0 
            prev_delta_max = 0
            prev_delta_vector = 0 
            prev_wick = 0
        else:
            prev_delta = abs(float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            prev_delta_max = abs(float(prev_candle['mid']['l']) - float(prev_candle['mid']['h']))
            prev_delta_vector = (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o']))
            
            if (float(prev_candle['mid']['c']) - float(prev_candle['mid']['o'])) > 0:
                prev_wick = float(prev_candle['mid']['h']) - float(prev_candle['mid']['c'])
            else:
                prev_wick = float(prev_candle['mid']['l']) - float(prev_candle['mid']['o'])


        lst.append([candle['time'],candle['volume'],float(candle['mid']['o']),float(candle['mid']['h']),float(candle['mid']['l']),
                        float(candle['mid']['c']),abs(float(candle['mid']['c']) - float(candle['mid']['o'])),
                    abs(float(candle['mid']['l']) - float(candle['mid']['h'])),
                     (float(candle['mid']['c']) - float(candle['mid']['o'])),max_vector,

                    prev_delta,prev_delta_max,prev_delta_vector,prev_wick

                   ])
        
        prev_candle = candle
        i += 1


    df = pd.DataFrame(lst,columns = ['time','volume','o','h','l','c','delta','delta_max',
                                     'delta_vector','max_vector','prev_delta',
                                     'prev_delta_max','prev_delta_vector','prev_wick'])
    
    
    df['prev_5'] = 0
    df['prev_10'] = 0
    df['prev_24'] = 0
    
    
    df['next_candle'] = 0
    df['next_2'] = 0
    df['next_3'] = 0
    df['next_4'] = 0
    df['next_5'] = 0
    df['next_10'] = 0
    
    df['next_candle+1'] = 0
    df['next_2+1'] = 0
    df['next_3+1'] = 0
    df['next_4+1'] = 0
    df['next_5+1'] = 0
    df['next_10+1'] = 0
    df['next_20+1'] = 0
        
    tot_length = df.shape[0]
    for i in range(df.shape[0] - 21):
        
        df['prev_5'].iloc[i] = df['c'].iloc[i] - df['c'].iloc[i - 5] 
        df['prev_10'].iloc[i] = df['c'].iloc[i] - df['c'].iloc[i - 10] 
        df['prev_24'].iloc[i] = df['c'].iloc[i] - df['c'].iloc[i - 24]    
        
        df['next_candle'].iloc[i] = df['c'].iloc[i + 1] - df['c'].iloc[i]
        df['next_2'].iloc[i] = df['c'].iloc[i + 2] - df['c'].iloc[i]
        df['next_3'].iloc[i] = df['c'].iloc[i + 3] - df['c'].iloc[i]
        df['next_4'].iloc[i] = df['c'].iloc[i + 4] - df['c'].iloc[i]
        df['next_5'].iloc[i] = df['c'].iloc[i + 5] - df['c'].iloc[i]
        df['next_10'].iloc[i] = df['c'].iloc[i + 10] - df['c'].iloc[i]
        
        df['next_candle+1'].iloc[i] = df['c'].iloc[i + 1+1] - df['c'].iloc[i+1]
        df['next_2+1'].iloc[i] = df['c'].iloc[i + 2+1] - df['c'].iloc[i+1]
        df['next_3+1'].iloc[i] = df['c'].iloc[i + 3+1] - df['c'].iloc[i+1]
        df['next_4+1'].iloc[i] = df['c'].iloc[i + 4+1] - df['c'].iloc[i+1]
        df['next_5+1'].iloc[i] = df['c'].iloc[i + 5+1] - df['c'].iloc[i+1]
        df['next_10+1'].iloc[i] = df['c'].iloc[i + 10+1] - df['c'].iloc[i+1]
        df['next_20+1'].iloc[i] = df['c'].iloc[i + 20+1] - df['c'].iloc[i+1] 
        
        
    col = test_col
    filter_df = df[df[col] >= np.percentile(df[col], percentile_test)]
    print('SHAPE',filter_df.shape)
    df['trigger'] = 0

    df.loc[df[df[col] >= np.percentile(df[col], percentile_test)].index,'trigger'] = 1    
    
    print('SHAPE2',df[df.trigger == 1].shape)
        
    return df
accountID='101-001-20062555-002'
access_token='2ad78612c58e604890fd961550f73cfd-28190cc09d5cbc67b2c14503575d6132'

api = API(access_token)
client=API(access_token)

jpy_pairs=['GBP_JPY','USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY']
major_pairs=['GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD','NZD_USD']
cross_pairs=['AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF','EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']
all_pairs=['USD_JPY','EUR_JPY','AUD_JPY','CAD_JPY','CHF_JPY',
           'GBP_USD','USD_CAD','USD_CHF','EUR_USD','AUD_USD',
           'NZD_USD','AUD_CAD','AUD_CHF','AUD_NZD','CAD_CHF',
           'EUR_AUD','EUR_CAD','EUR_CHF','NZD_CAD','NZD_CHF']



api



In [13]:
def check_price(pair):
    bid_ask=[]
    params={"instruments":pair}
            
    r = pricing.PricingInfo(accountID=accountID, params=params)
    a=api.request(r)
    
    bid=float(a['prices'][0]['bids'][0]['price'])
    ask=float(a['prices'][0]['asks'][0]['price'])
    avg=(bid+ask)/2
    
    bid_ask.append(bid)
    bid_ask.append(ask)
    bid_ask.append(avg)
    
    
    
    return bid_ask
pair = 'XAU_USD'
check_price(pair)

[1647.716, 1648.174, 1647.945]

# 100000 units = 1 lot

# OPEN TRADE

In [16]:
pair = 'XAU_USD'
pair = 'USD_JPY'
data = {
  "order": {
    "units": "-1000",
    "instrument": pair,
    "timeInForce": "FOK",
    "type": "MARKET",
    "positionFill": "DEFAULT"
  }
}

r = orders.OrderCreate(accountID, data=data)
client.request(r)
print(r.response)

{'orderCreateTransaction': {'id': '6', 'accountID': '101-001-20062555-002', 'userID': 20062555, 'batchID': '6', 'requestID': '43022130875256765', 'time': '2022-11-01T22:12:01.404042813Z', 'type': 'MARKET_ORDER', 'instrument': 'USD_JPY', 'units': '-1000', 'timeInForce': 'FOK', 'positionFill': 'DEFAULT', 'reason': 'CLIENT_ORDER'}, 'orderFillTransaction': {'id': '7', 'accountID': '101-001-20062555-002', 'userID': 20062555, 'batchID': '6', 'requestID': '43022130875256765', 'time': '2022-11-01T22:12:01.404042813Z', 'type': 'ORDER_FILL', 'orderID': '6', 'instrument': 'USD_JPY', 'units': '-1000', 'requestedUnits': '-1000', 'price': '148.330', 'pl': '0.0000', 'quotePL': '0', 'financing': '0.0000', 'baseFinancing': '0', 'commission': '0.0000', 'accountBalance': '100000.0000', 'gainQuoteHomeConversionFactor': '0.00670754113', 'lossQuoteHomeConversionFactor': '0.006774953604', 'guaranteedExecutionFee': '0.0000', 'quoteGuaranteedExecutionFee': '0', 'halfSpreadCost': '0.0708', 'fullVWAP': '148.330'

In [20]:
pair = 'XAU_USD'
#pair = 'USD_JPY'
data = {
  "order": {
    "units": "-1000",
    "instrument": pair,
    "timeInForce": "FOK",
    "type": "MARKET",
    "positionFill": "DEFAULT"
  }
}

r = orders.OrderCreate(accountID, data=data)
client.request(r)
print(r.response)

V20Error: {"orderRejectTransaction":{"id":"10","accountID":"101-001-20062555-002","userID":20062555,"batchID":"10","requestID":"43022133291938071","time":"2022-11-01T22:21:37.471836292Z","type":"MARKET_ORDER_REJECT","rejectReason":"INSTRUMENT_NOT_TRADEABLE","instrument":"XAU_USD","units":"-1000","timeInForce":"FOK","positionFill":"DEFAULT","reason":"CLIENT_ORDER"},"relatedTransactionIDs":["10"],"lastTransactionID":"10","errorMessage":"The instrument specified is not tradeable by the Account","errorCode":"INSTRUMENT_NOT_TRADEABLE"}

# CLOSE TRADE

In [19]:
data ={
          "longUnits": "ALL",
          "shortUnits": "ALL"
        }
    
r = positions.PositionClose(accountID=accountID,
                             instrument=pair,
                             data=data)
print(client.request(r))

V20Error: {"longOrderRejectTransaction":{"id":"8","accountID":"101-001-20062555-002","userID":20062555,"batchID":"8","requestID":"43022132083596079","time":"2022-11-01T22:16:49.113001508Z","type":"MARKET_ORDER_REJECT","rejectReason":"CLOSEOUT_POSITION_DOESNT_EXIST","instrument":"USD_JPY","timeInForce":"FOK","positionFill":"REDUCE_ONLY","reason":"POSITION_CLOSEOUT","longPositionCloseout":{"instrument":"USD_JPY","units":"ALL"}},"shortOrderRejectTransaction":{"id":"9","accountID":"101-001-20062555-002","userID":20062555,"batchID":"8","requestID":"43022132083596079","time":"2022-11-01T22:16:49.113001508Z","type":"MARKET_ORDER_REJECT","rejectReason":"CLOSEOUT_POSITION_REJECT","instrument":"USD_JPY","units":"1000","timeInForce":"FOK","positionFill":"REDUCE_ONLY","reason":"POSITION_CLOSEOUT","shortPositionCloseout":{"instrument":"USD_JPY","units":"ALL"}},"relatedTransactionIDs":["8","9"],"lastTransactionID":"9","errorMessage":"The Position requested to be closed out does not exist","errorCode

In [2]:
!pip install pymt5

In [3]:
!pip install metaapi-cloud-sdk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.8/228.8 kB 1.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 648.3/648.3 kB 3.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.7/88.7 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.8/84.8 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.4/68.4 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━

In [4]:
from metaapi_cloud_sdk import MetaApi, CopyFactory

In [ ]:
    
def majors_trade(sl,tp,pair,current_price,wick,rev):
    bal,marg,trades=account_value()
    units=str(int(units_calc(bal,.01,sl)))
    stoploss=str(stoploss_calc(pair,wick,sl,rev))
    takeprofit=str(takeprofit_calc(pair,current_price,tp,rev))
    
    order={"order": 
        {
        "type": "MARKET",
        "units": units,
        "instrument": pair,
        "timeInForce": "FOK",
        "takeProfitOnFill": {
            "timeInForce": "GTC",
            "price": takeprofit
        },
        "stopLossOnFill": {
            "timeInForce": "GTC",
            "price": stoploss}
        }
        }
    r=orders.OrderCreate(accountID,data=order)
    api.request(r)  

In [ ]:
def account_value():
        
    api = oandapyV20.API(access_token=access_token)
    r = accounts.AccountDetails(accountID)
    acc_details=api.request(r)
    
    balance=float(acc_details['account']['balance'])
    margin_available=float(acc_details['account']['marginAvailable'])
    open_trades=float(acc_details['account']['openTradeCount'])
    
    return balance,margin_available,open_trades


def units_calc(trading_account,max_risk,sl):
    units=(max_risk*trading_account/sl)*10000
    
    return units

def stoploss_calc(pair,wick,sl,rev):
    sl_price=0
    if 'JPY' in pair:
        sl=sl*.01
        if rev==1:
            sl_price=wick-sl
        elif rev==2:
            sl_price=wick+sl
    else:
        sl=sl*.0001
        if rev==1:
            sl_price=wick-sl
        elif rev==2:
            sl_price=wick+sl    
    return sl_price
    
def takeprofit_calc(pair,current_price,tp,rev):
    tp_price=0
    if 'JPY' in pair:
        tp=tp*.01
        if rev==1:
            tp_price=current_price+tp
        elif rev==2:
            tp_price=current_price-tp
    else:
        tp=tp*.0001
        if rev==1:
            tp_price=current_price+tp
        elif rev==2:
            tp_price=current_price-tp  
    return tp_price


def jpy_trade(sl,tp,pair,current_price,wick,rev):
    bal,marg,trades=account_value()
    units=str(int(units_calc(bal,.01,sl)))
    stoploss=str(stoploss_calc(pair,wick,sl,rev))
    takeprofit=str(takeprofit_calc(pair,current_price,tp,rev))
    
    order={"order": 
        {
        "type": "MARKET",
        "units": units,
        "instrument": pair,
        "timeInForce": "FOK",
        "takeProfitOnFill": {
            "timeInForce": "GTC",
            "price": takeprofit
        },
        "stopLossOnFill": {
            "timeInForce": "GTC",
            "price": stoploss}
        }
        }
    r=orders.OrderCreate(accountID,data=order)
    api.request(r)
    

    
def majors_trade(sl,tp,pair,current_price,wick,rev):
    bal,marg,trades=account_value()
    units=str(int(units_calc(bal,.01,sl)))
    stoploss=str(stoploss_calc(pair,wick,sl,rev))
    takeprofit=str(takeprofit_calc(pair,current_price,tp,rev))
    
    order={"order": 
        {
        "type": "MARKET",
        "units": units,
        "instrument": pair,
        "timeInForce": "FOK",
        "takeProfitOnFill": {
            "timeInForce": "GTC",
            "price": takeprofit
        },
        "stopLossOnFill": {
            "timeInForce": "GTC",
            "price": stoploss}
        }
        }
    r=orders.OrderCreate(accountID,data=order)
    api.request(r)    
    
    
    

def cross_trade(sl,tp,pair,current_price,wick,rev):
    bal,marg,trades=account_value()
    units=str(int(units_calc(bal,.01,sl)))
    stoploss=str(stoploss_calc(pair,wick,sl,rev))
    takeprofit=str(takeprofit_calc(pair,current_price,tp,rev))
    
    order={"order": 
        {
        "type": "MARKET",
        "units": units,
        "instrument": pair,
        "timeInForce": "FOK",
        "takeProfitOnFill": {
            "timeInForce": "GTC",
            "price": takeprofit
        },
        "stopLossOnFill": {
            "timeInForce": "GTC",
            "price": stoploss}
        }
        }
    r=orders.OrderCreate(accountID,data=order)
    api.request(r)

In [ ]:
trade_open = 1




In [ ]:
accountID='101-001-10831130-001'
access_token='45cc721fe40759fb0b76d015456ba8b8-27c86888ddbab88249c3914ba1ff2a01'

api = API(access_token)